In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, get_scheduler
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm
import re, hashlib

In [3]:
fake = pd.read_csv('/kaggle/input/fake-news-detection-datasets/News _dataset/Fake.csv')
real = pd.read_csv('/kaggle/input/fake-news-detection-datasets/News _dataset/True.csv')

In [4]:
fake['label'] = 0
real['label'] = 1
data = pd.concat([fake, real], axis=0).reset_index(drop=True)

In [5]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'\b(reuters|ap|cnn|bbc|fox|nbc|abc|cbs)\b', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

data['text'] = data['text'].apply(clean_text)
data = data[data['text'].str.len() > 10].reset_index(drop=True)

In [6]:
def get_text_hash(text):
    normalized = re.sub(r'[^\w\s]', '', text.lower())
    normalized = re.sub(r'\s+', ' ', normalized).strip()
    return hashlib.md5(normalized.encode()).hexdigest()

data = data.drop_duplicates(subset='text').reset_index(drop=True)
data['text_hash'] = data['text'].apply(get_text_hash)
data = data.drop_duplicates(subset='text_hash').reset_index(drop=True)
data = data.drop('text_hash', axis=1)
data = data[(data['text'].str.len() >= 50) & (data['text'].str.len() <= 5000)].reset_index(drop=True)

In [7]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    data['text'].tolist(), data['label'].tolist(),
    test_size=0.3, random_state=42, stratify=data['label']
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels,
    test_size=0.5, random_state=42, stratify=temp_labels
)


In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)
test_dataset = NewsDataset(test_encodings, test_labels)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
gradient_accumulation_steps = 2 
num_training_steps = (num_epochs * len(train_loader)) // gradient_accumulation_steps
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=50, num_training_steps=num_training_steps)

In [12]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss, preds, true_labels = 0, [], []
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            total_loss += outputs.loss.item()
            preds.extend(torch.argmax(outputs.logits, dim=-1).cpu().numpy())
            true_labels.extend(batch['labels'].cpu().numpy())
    return total_loss / len(dataloader), accuracy_score(true_labels, preds), preds, true_labels

In [ ]:
best_val_accuracy = 0
patience, patience_counter = 2, 0 

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}")

    for step, batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss / gradient_accumulation_steps
        loss.backward()
        total_train_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})

    avg_train_loss = total_train_loss / len(train_loader)
    val_loss, val_accuracy, _, _ = evaluate_model(model, val_loader)

    print(f"\nEpoch {epoch+1}: Train Loss={avg_train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_accuracy:.4f}")

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), "best_model.pt")
        patience_counter = 0
        print("Saved best model")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

In [14]:
torch.save(model.state_dict(), "bestmodel.pt")


In [ ]:
model.save_pretrained("bestmodel")
tokenizer.save_pretrained("bestmodel")


In [ ]:
model.load_state_dict(torch.load("bestmodel.pt"))
test_loss, test_accuracy, preds, true_labels = evaluate_model(model, test_loader)

print("\n" + "="*50)
print("FINAL TEST RESULTS")
print("="*50)
print(f"Test Accuracy: {test_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(true_labels, preds))

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DistilBertForSequenceClassification.from_pretrained("bestmodel").to(device)
tokenizer = DistilBertTokenizerFast.from_pretrained("bestmodel")
model.eval()


In [18]:
import pandas as pd, numpy as np, re
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score

def clean_text(text):
    if pd.isna(text): return ""
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"\S+@\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

class LiarDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(
            texts, truncation=True, padding=True, max_length=max_length
        )
        self.labels = labels
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(int(self.labels[idx]), dtype=torch.long)
        return item

@torch.no_grad()
def evaluate_loader(model, dataloader, device):
    model.eval()
    preds, trues = [], []
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        out = model(**batch)
        preds.extend(torch.argmax(out.logits, dim=-1).cpu().numpy())
        trues.extend(batch["labels"].cpu().numpy())
    acc = accuracy_score(trues, preds)
    return acc, np.array(preds), np.array(trues)


In [ ]:
train_df = pd.read_csv("/kaggle/input/liar-dataset/train.tsv", sep="\t", header=None)
val_df   = pd.read_csv("/kaggle/input/liar-dataset/valid.tsv", sep="\t", header=None)
test_df  = pd.read_csv("/kaggle/input/liar-dataset/test.tsv", sep="\t", header=None)

print(train_df.head())


In [ ]:

for df in [train_df, val_df, test_df]:
    df.rename(columns={1: "label", 2: "text"}, inplace=True)

label_map = {
    "pants-fire": 0,
    "false": 0,
    "barely-true": 0,
    "half-true": 1,
    "mostly-true": 1,
    "true": 1
}
for df in [train_df, val_df, test_df]:
    df["label"] = df["label"].map(label_map)

for df in [train_df, val_df, test_df]:
    df.dropna(subset=["label"], inplace=True)

def clean_text(text):
    return str(text).strip().lower()

for df in [train_df, val_df, test_df]:
    df["text"] = df["text"].apply(clean_text)

print(train_df.head())


In [ ]:
from transformers import DistilBertForSequenceClassification


liar_model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", 
    num_labels=2
)


liar_model.load_state_dict(torch.load("bestmodel.pt", map_location=device))
liar_model.to(device)
liar_model.eval()


In [24]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_dataset(df):
    return tokenizer(
        df["text"].tolist(),
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )

train_encodings_liar = tokenize_dataset(train_df)
val_encodings_liar   = tokenize_dataset(val_df)
test_encodings_liar  = tokenize_dataset(test_df)


In [25]:
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset_liar = NewsDataset(train_encodings_liar, train_df["label"].tolist())
val_dataset_liar   = NewsDataset(val_encodings_liar, val_df["label"].tolist())
test_dataset_liar  = NewsDataset(test_encodings_liar, test_df["label"].tolist())


In [26]:
from torch.utils.data import DataLoader

train_loader_liar = DataLoader(train_dataset_liar, batch_size=16, shuffle=False)
val_loader_liar   = DataLoader(val_dataset_liar, batch_size=16, shuffle=False)
test_loader_liar  = DataLoader(test_dataset_liar, batch_size=16, shuffle=False)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

def evaluate_model(model, data_loader):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            preds.extend(predictions.cpu().numpy())
            labels.extend(batch["labels"].cpu().numpy())

    acc = accuracy_score(labels, preds)
    report = classification_report(labels, preds)
    return acc, report

# Evaluate
acc, report = evaluate_model(liar_model, test_loader_liar)

print("=== LIAR Test Results ===")
print("Accuracy:", acc)
print("Report:\n", report)


In [ ]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.load_state_dict(torch.load("bestmodel.pt"))
model.to(device)


In [29]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_data(df, tokenizer, max_len=128):
    return tokenizer(
        df["text"].tolist(),
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )

train_encodings = tokenize_data(train_df, tokenizer)
val_encodings   = tokenize_data(val_df, tokenizer)
test_encodings  = tokenize_data(test_df, tokenizer)


In [30]:
import torch
from torch.utils.data import TensorDataset, DataLoader

def make_loader(encodings, labels, batch_size=16, shuffle=False):
    dataset = TensorDataset(
        encodings["input_ids"], 
        encodings["attention_mask"], 
        torch.tensor(labels.values)
    )
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

train_loader = make_loader(train_encodings, train_df["label"], shuffle=True)
val_loader   = make_loader(val_encodings, val_df["label"])
test_loader  = make_loader(test_encodings, test_df["label"])


In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm
import torch


optimizer = AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)

num_epochs = 10  
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def evaluate(model, val_loader):
    
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    criterion = torch.nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for batch in val_loader:
            batch = [t.to(device) for t in batch]
            input_ids, attention_mask, labels = batch

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()
            preds = torch.argmax(logits, dim=-1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    return val_loss / len(val_loader), correct / total



best_val_acc = 0
patience, patience_counter = 2, 0  

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    
    for batch in train_loader:
        batch = [t.to(device) for t in batch]
        input_ids, attention_mask, labels = batch

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        progress_bar.update(1)

    avg_train_loss = total_loss / len(train_loader)
    val_loss, val_acc = evaluate(model, val_loader)

    print(f"Epoch {epoch+1}: Train Loss={avg_train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), "bestmodel.pt")  
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break


model.load_state_dict(torch.load("bestmodel.pt"))


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

def evaluate(model, loader):
    model.eval()
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = [t.to(device) for t in batch]
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            preds.extend(predictions.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(true_labels, preds)
    report = classification_report(true_labels, preds)
    return acc, report

acc, report = evaluate(model, test_loader)
print("Fine-tuned Test Accuracy:", acc)
print(report)
